# Visualizing flight paths to Chicago

starting from here : https://makersportal.com/blog/2018/8/23/airport-destination-mapper-in-real-time-with-python

airport data from here : https://ourairports.com/help/data-dictionary.html 

I wonder if I should grab all the flight info that I can, and just highlight the chicago ones?  That might look cooler.  I could probably also remove the map if I get enough.

In [2]:
import numpy as np
import pandas as pd

import requests
import re 

from dateutil.parser import parse
from datetime import *

import cartopy 
import cartopy.crs as ccrs

import matplotlib.pyplot as plt
import matplotlib


%matplotlib inline

In [22]:
#file from ourairports.com
airports_df = pd.read_csv('airports.csv')
airports_df

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.949200,-151.695999,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
4,6526,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,NaN,NaN,NaN,NaN,NaN,00AR
5,322127,00AS,small_airport,Fulton Airport,34.942803,-97.818019,1100.0,NaN,US,US-OK,Alex,no,00AS,NaN,00AS,NaN,NaN,NaN
6,6527,00AZ,small_airport,Cordes Airport,34.305599,-112.165001,3810.0,NaN,US,US-AZ,Cordes,no,00AZ,NaN,00AZ,NaN,NaN,NaN
7,6528,00CA,small_airport,Goldstone /Gts/ Airport,35.350498,-116.888000,3038.0,NaN,US,US-CA,Barstow,no,00CA,NaN,00CA,NaN,NaN,NaN
8,324424,00CL,small_airport,Williams Ag Airport,39.427188,-121.763427,87.0,NaN,US,US-CA,Biggs,no,00CL,NaN,00CL,NaN,NaN,NaN
9,322658,00CN,heliport,Kitchen Creek Helibase Heliport,32.727374,-116.459742,3350.0,NaN,US,US-CA,Pine Valley,no,00CN,NaN,00CN,NaN,NaN,NaN


In [23]:
def convertTime(tlist, today = datetime(2020, 2, 6)):
    out = []
    for x in tlist :
        val = x[0].replace('\xa0',' ')
        try:
            dt = parse(val, default = today)
            out.append(dt.hour + dt.minute/60. + dt.second/3600.)
        except:
            out.append(val)
            #out.append(np.nan)
    return out

def getIdent(alist):
    out = []
    lat = []
    lon = []
    for x in alist:
        try:
            p1 = x[0].rfind('(')
            p2 = x[0].rfind(')')
            val = x[0][p1+1:p2]
            v1 = val
            v2 = val
            p3 = val.find('/')
            if (p3 != -1):
                v1 = val[:p3]
                v2 = val[p3+2:]
            #out.append(val)
            out.append(x[0])
            row = airports_df.loc[(airports_df['ident'] == v1) | 
                                 (airports_df['gps_code'] == v1) |
                                 (airports_df['iata_code'] == v1) | 
                                 (airports_df['local_code'] == v1) |
                                 (airports_df['ident'] == v2) | 
                                 (airports_df['gps_code'] == v2) |
                                 (airports_df['iata_code'] == v2) | 
                                 (airports_df['local_code'] == v2) ]
            if (len(row) >=1):
                lat.append(row['latitude_deg'].values[0])
                lon.append(row['longitude_deg'].values[0])
            else:
                lat.append(-999.)
                lon.append(-999.)
        except:
            out.append('foo')
            lat.append(-999)
            lon.append(-999)
    return out, lon, lat

key_match_start = '<table class="prettyTable fullWidth"'
key_match_end = '</table>'
              

In [24]:
offset = 0
maxOffset = 10000
origin = []
lat = []
lon = []
arrive = []
depart = []
while offset < maxOffset:
    #for requests to Flight Aware
    page = requests.get('https://flightaware.com/live/airport/KORD/arrivals?;offset='+str(offset)+';order=actualarrivaltime;sort=DES')
    p1 = page.text.find(key_match_start)
    p2 = page.text.rfind(key_match_end)
    flight_html = page.text[p1:p2]

    flight_df = pd.read_html(flight_html)[0]
    o, lo, la = getIdent(flight_df.iloc[:, flight_df.columns.get_level_values(1)=='Origin'].values)
    d = convertTime(flight_df.iloc[:, flight_df.columns.get_level_values(1)=='Departure'].values)
    a = convertTime(flight_df.iloc[:, flight_df.columns.get_level_values(1)=='Arrival'].values)
    
    origin.extend(o)
    lon.extend(lo)
    lat.extend(la)
    depart.extend(d)
    arrive.extend(a)
    
    offset += 20
    print(offset)

              

/Users/ageller/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1206: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/Users/ageller/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1206: UnknownTimezoneWarning: tzname PST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/Users/ageller/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1206: UnknownTimezoneWarning: tzname MST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


20
40
60
80


/Users/ageller/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1206: UnknownTimezoneWarning: tzname AST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


100
120
140
160


/Users/ageller/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1206: UnknownTimezoneWarning: tzname CET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


...

920
940
960
980


/Users/ageller/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1206: UnknownTimezoneWarning: tzname IST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


1000
1020
1040


/Users/ageller/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1206: UnknownTimezoneWarning: tzname HST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460
2480
2500
2520
2540
2560
2580
2600
2620
2640
2660
2680
2700
2720
2740
2760
2780
2800
2820
2840
2860
2880
2900
2920
2940
2960
2980
3000
3020
3040
3060
3080
3100
3120
3140
3160
3180
3200
3220
3240
3260
3280
3300
3320
3340
3360
3380
3400
3420
3440
3460
3480
3500
3520
3540
3560
3580
3600
3620
3640
3660
3680
3700
3720
3740
3760
3780
3800
3820
3840
3860
3880
3900
3920
3940
3960
3980
4000
4020
4040
4060
4080
4100
4120
4140
4160
4180
4200
4220
4240
4260
4280
4300
4320
4340
4360
4380
4400
4420
4440
4460
4480
4500
4520
4540
4560
4580
4600
4620
4640
4660
4680
4700
4720
4740
4760
4780
4800
4820
4840
4860
4880
4900
4920
4940
4960
4980
5000
5020
5040


In [25]:
print(len(origin), len(lon), len(lat), len(arrive), len(depart))

9966 9966 9966 9966 9966


In [26]:
out_df = pd.DataFrame()
out_df['origin'] = origin
out_df['longitude'] = lon
out_df['latitude'] = lat
out_df['departure'] = depart
out_df['arrival'] = arrive
out_df

,origin,longitude,latitude,departure,arrival
0,Reagan National (KDCA),-77.037697,38.852100,19.5333,20.0667
1,Boston Logan Intl (KBOS),-71.005203,42.364300,19,20.0333
2,Raleigh-Durham Intl (KRDU),-78.787498,35.877602,19.3667,20.0333
3,Palm Springs Intl (KPSP),-116.507004,33.829700,14.6667,20.0167
4,Montreal-Trudeau (CYUL),-73.740799,45.470600,19.0833,20.0167
5,Jackson-Medgar Wiley Evers Intl (KJAN),-90.075897,32.311199,18.25,19.9833
6,Hartsfield-Jackson Intl (KATL),-84.428101,33.636700,19.4833,19.9833
7,Newark Liberty Intl (KEWR),-74.168701,40.692501,19.15,19.9833
8,LaGuardia (KLGA),-73.872597,40.777199,19.0833,19.9667
9,Des Moines Intl (KDSM),-93.663101,41.534000,18.9833,19.95


In [27]:
out_df.to_csv('flights.csv', index=False)

### Plot these with plotFlights.py